#### The following are example SQL interview questions from "Ace the Data Science Interview" by Kevin Huo and Nick Singh. All credit to the authors. 

8.1 FACEBOOK: Assume you have the below events table on app analytics. Write a query to get the clickthrough rate per app in 2019.

events:
| column_name | type |
|-------------|------|
| app_id | integer|
| event_id | string('impression', 'click') |
| timestamp | datetime |

In [ ]:
-----------------
-- MY SOLUTION --
-----------------

-- first question: how many impressions were there
-- filter by date in 2019
-- group by app_id

SELECT COUNT(event_id)
  FROM events
  GROUP BY app_id
  HAVING event_id = 'impression'
     AND timestamp BETWEEN 2019-01-01 AND 2019-12-31

-- second quetion: how many clicks were there
-- filter by date in 2019
-- group by app_id

SELECT COUNT(event_id)
  FROM events 
  GROUP BY app_id
  HAVING event_id = 'click'
     AND timestamp BETWEEN 2019-01-01 AND 2019-12-31

-- put it together: get the rate of clicks per event

SELECT app_id, 
       (
        COUNT(event_id) / (
                            SELECT COUNT(event_id)
                                FROM events
                                GROUP BY app_id
                                HAVING event_id = 'impression'
                                    AND timestamp BETWEEN '2019-01-01' AND '2019-12-31'
                            )
        )
  FROM events 
  GROUP BY app_id
  HAVING event_id = 'click'
     AND timestamp BETWEEN '2019-01-01' AND '2019-12-31'
;

-- this solution did not work because the subqueries
-- result in more than one row (because there is more than one app_id).  
-- you can't divide by multiple rows at the same time like you might be able to in a pandas
-- df. 

In [ ]:
-------------------
-- BOOK SOLUTION --
-------------------

SELECT 
  app_id,
  SUM(IF(event_id = 'click', 1, 0)) / SUM(IF(event_id = 'impression', 1, 0))
    AS ctr
FROM 
  events 
WHERE 
  timestamp >= '2019-01-01'
  AND timestamp <= '2020-01-01'
GROUP BY 
  1 -- (this groups by the first column in the SELECT list, equivalent to `GROUP BY app_id`
  

8.2 ROBINHOOD Assume you are given the tables below containing information on trades and users. Write a query to list the top three cities that had the most number of completed orders. 

trades:
|column_name|type|
|-----------|----|
| order_id | integer|
|user_id|integer|
|price|float|
|quantity|integer|
|status|string('complete, cancelled')|
|timestamp|datetime|

users:
|column_name|type|
|---|----|
|user_id|integer|
|city|string|
|email|string|
|signup_date| datetime|


In [ ]:
-----------------
-- MY SOLUTION --
-----------------

SELECT u.city, COUNT(DISTINCT t.order_id) AS n_orders
  FROM trades t
    JOIN users u USING(user_id)
  WHERE t.status = 'complete'
  GROUP BY u.city
  ORDER BY n_orders DESC
  LIMIT 3
;

In [ ]:
-------------------
-- BOOK SOLUTION --
-------------------

SELECT 
  u.city, 
  COUNT(DISTINCT t.order_id) AS num_orders
FROM 
  trades t
  JOIN users u ON t.user_id = u.user_id
WHERE 
  t.status = 'complete'
GROUP BY 
  city
ORDER BY 
  num_orders DESC
LIMIT 
  3

8.3 NEW YORK TIMES: Assume that you are given the table below containing information on viewership by device type (where the three types are laptop tablet, and phone). Define "mobile" as the sum of tablet and phone viewership numbers. Write a query to compare the viewership on laptops versus mobile devices. 

viewership:
| column_name | type |
|-------------|------|
|user_id|integer|
|device_type| string ('laptop', 'tablet', 'phone')|
|view_time | datetime|

In [ ]:
--------------
-- SOLUTION --
--------------

SELECT 
  SUM(IF(device_type IN ('phone', 'tablet'), 1, 0)) as mobile_views,
  SUM(IF(device_type = 'laptop', 1, 0)) as laptop_views
FROM 
  viewership
;

8.4 AMAZON: Assume you are given the table below for spending activity by product type. Write a query to calculate the cumulative spend so far by date for each product over time in chronological order. 

total_trans:
|column_name|type|
|-----------|----|
|order_id|integer|
|user_id|integer|
|product_id|string|
|spend|float|
|trans_date|datetime|

In [ ]:
-- this solution requires a window function, with which i am not familiar. I will come back to this later.

8.5 EBAY: Assume that you are given the table below containing information on various orders made by customers. Write a query to obtain the names of the ten customers who have ordered the highest number of products among those customers who have spent at least $1000 in total. 

user_transactions:
|column_name|type|
|--|--|
|transaction_id|integer|
|product_id|integer|
|user_id|integer|
|spend|float|
|trans_date|datetime|


In [ ]:
--------------
-- SOLUTION --
--------------

SELECT user_id, 
       COUNT(product_id) AS n_products
  FROM user_transactions
  GROUP BY user_id
  HAVING SUM(spend) > 1000
  ORDER BY n_products DESC
  LIMIT 10
;

8.6 TWITTER: Assume you are given the table below containing information on tweets. Write a query to obtain a histogram of tweets posted per user in 2020. 

tweets:
|column_name|type|
|--|--|
|tweet_id|integer|
|user_id|integer|
|msg|string|
|tweet_date|datetime|

8.7 STITCH FIX: Assume you are given the table below containing information on user purchases. Write a query to obtain the number of people who purchased at least one or more of the same product on multiple days. 

purchases:
|column_name|type|
|--|--|
|purchase_id|integer|
|user_id|integer|
|product_id|integer|
|quantity|integer|
|price|float|
|purchase_time|datetime|


In [ ]:
-----------------
-- MY SOLUTION --
-----------------

SELECT user_id, 
       COUNT(DISTINCT user_id) 
  FROM purchases
  WHERE user_id IN (
                    SELECT user_iD
                      FROM ( 
                            SELECT user_id, 
                                product_id,
                                date(datefield), 
                                COUNT(*) as n_purchases
                            FROM purchases
                            GROUP BY user_id, 
                                    product_id,
                                    date(datefield)
                            HAVING n_purchases > 1
                            ) t
                    )
  GROUP BY user_id
   

In [ ]:
-------------------
-- BOOK SOLUTION --
-------------------

SELECT COUNT(DISTINCT user_id)
  FROM (
        SELECT user_id, 
               RANK() OVER (
                            PARTITION BY user_id,
                                         product_id,
                            ORDER BY CAST(purchase_time AS DATE)
                           ) AS purchase_no
          FROM purchases
        ) t 
  WHERE purchase_no = 2